In [ ]:
import numpy as np
import copy
import torch
import itertools
import matplotlib.pyplot as plt
from torch.distributions import MultivariateNormal, Normal
from torch.optim.lr_scheduler import LambdaLR, ConstantLR, CyclicLR
from torch.autograd import Variable
from scipy.optimize import root_scalar, shgo

device = "cpu"
default_tensor_type = torch.FloatTensor

# Problem Setting

The Objective of this notebook is to study the compromise Performance vs Robustness of Statistics over distributions of Rankings

We need to define an indexation order on $\mathfrak{S}_n$ in order to write the loss as a matrix in $\mathbb{R}^{n! \times n!}$ and distributions as vectors in $\mathbb{R}^{n!}$.

In [ ]:
n_items = 4
all_ranks = list(itertools.permutations(list(np.arange(n_items))))
all_ranks = [np.array(elem) for elem in all_ranks]
torch_all_ranks = torch.from_numpy(np.asarray(all_ranks))

**DEFINITION:** A statistics over a distribution of rankings is a function $T:\Delta^{\mathfrak{S}_n} \to \mathfrak{S}_n$.

**PERFORMANCE** as defined by 
$$
p \mapsto \mathbb{E}_{Y\sim p}\left(\tau_K(Y, T(p))\right)
$$
where 
$\tau_K$ is the Kendall Tau:
$$
\tau_K(y, \sigma) = \sum_{i,j} \mathbb{1}\{\sigma^{-1}(i) < \sigma^{-1}(j)\}\mathbb{1}\{y^{-1}(i) > y^{-1}(j)\}
$$

In [ ]:
def pairwise_matrix(p_torch, torch_all_ranks, n=4):
    M = torch.zeros(n,n)
    for i in range(n):
        for j in range(i+1,n):
            idxs = torch.tensor([torch.argwhere(rk == i).item() < torch.argwhere(rk == j).item() for rk in torch_all_ranks])
            val = torch.sum(p_torch[0,idxs]).item()
            M[i,j] = val
            M[j,i] = 1-val
    return M


def expected_kendall(P1, P2):
    return torch.norm(P1 * (1-P2), 1)
    #return torch.sum(torch.triu(P1,1)*torch.triu(1-P2,1) + torch.triu(P2,1)*torch.triu(1-P1,1))

**ROBUSTNESS** as defined by the "breakdown function" of T:
$$
\varepsilon^*(\delta,p,T) = \inf \left\{\varepsilon>0 : \sup_{||p-q|| \leq \varepsilon} \rho_{\tau_K}(T(p), T(q)) \geq \delta \right\}
$$
where
$$
\rho_{\tau_K} = H_K^{(1/2)}(\pi_1, \pi_2) := \frac{1}{2}\left(\max_{\sigma_1 \in \pi_1} \min_{\sigma_2 \in \pi_2} \tau_K(\sigma_1, \sigma_2) + \max_{\sigma_2 \in \pi_2} \min_{\sigma_1 \in \pi_1} \tau_K(\sigma_1, \sigma_2) \right)
$$
The difficulty with the computation of the breakdown function is that the underlying optimization cannot be performed directly.

In [ ]:
def symmetrized_hausdorff_on_kendall(P1, P2):
    return torch.norm(torch.triu(P1,1) - torch.triu(P2,1), 1)

$$
\tilde{\rho}_{\tau_K} = H_K^{dis}(\pi_1, \pi_2) := \max_{\sigma_2 \in \pi_2} \min_{\sigma_1 \in \pi_1} \tau_K(\sigma_1, \sigma_2)
$$

In [ ]:
def disymmetrized_hausdorff_on_kendall(P1, P2):
    # If P2 is included in P1
    if (P1[P1 != 1/2] == P2[P1 != 1/2]).all():
        return torch.tensor(0.0)
    # If P1 is included in P2
    elif (P1[P2 != 1/2] == P2[P2 != 1/2]).all():
        return 2*torch.norm(torch.triu(P1,1) - torch.triu(P2,1), 1)
    else:
        idxs = torch.argwhere(P1 == 1/2)
        v = torch.sum(P2 == 1/2)/2
        idxs2 = torch.argwhere(P2 == 1/2)
        P1[[idxs[:, 0], idxs[:, 1]]] = 0
        P2[[idxs[:, 0], idxs[:, 1]]] = 0
        P1[[idxs2[:, 0], idxs2[:, 1]]] = 0
        P2[[idxs2[:, 0], idxs2[:, 1]]] = 0
        return torch.norm(torch.triu(P1,1) - torch.triu(P2,1), 1) + v

# Computing the breakdown function

We approximate the computation of $\varepsilon^*$ in the following way:

1. We find a distribution $\tilde q_{\varepsilon,\delta, p, T}$ that is a solution of the smoothed problem:
$$
\tilde q_{\varepsilon, p, T} = \arg\sup_{||p-q|| \leq \varepsilon} \phi(p,q)
$$
where $\phi(\cdot,q) = \rho_{\tau_K}(T(\cdot), T(q)) \star k(\cdot)$ with $k$ being a smoothing convolution kernel, here a multivariate Gaussian.
2. We approximate $\varepsilon^*$ by
$$
\tilde\varepsilon(\delta,p,T) = \inf \left\{\varepsilon>0 : \rho_{\tau_K}\left(T(p), T(\tilde q_{\varepsilon, p, T})\right) \geq \delta \right\}
$$

### Computing $\phi$

A method for smoothing a bivariate function $m:{\cal Y}\times {\cal X} \to \mathbb{R}$ on its second argument using a mean convolution with kernel $k$.

$$
\tilde m(y, x) = (m(y,\cdot)\star k)(x)
$$

In [ ]:
def smooth_pg_2(m, k_x):
    r"""
    Smooth a metric $m$ by convolution with a kernel $k_x$ centered in $x$.
    $$\tilde{m}(y,x) = \int m(y,u) k_x(u) du$$
    As the gradient is not available in closed-form, the loss build is a policy gradient loss, that leads to a noisy but
    unbiased estimate of the gradient of $\tilde{m}$.
    $$g_x(y,u) = \log(k_x(u)) f(y,u) ~~~~\text{for}~~~ u\sim k_x$$
    :param m: function to be smoothed
    :param k_x: function $x\mapsto k$ where $k$ is a kernel centered in $x$
    :return: $g_x(y,u)$ for $u\sim k_x$
    """
    def smoothed_m_pg(y, x):
        k = k_x(x)
        batch_size = torch.Size()
        if len(y.size()) > 1:
            batch_size = y.size()[:-1]
        u = k.sample(batch_size).to(device).type(default_tensor_type)
        loss = k.log_prob(u) * m(y,u)
        return loss
    
    return smoothed_m_pg

In [ ]:
def monte_carlo_phi(p, q, dist_Tp_Tq, std_dev_=0.00001, nb_simu=50):
    kernel_conv = lambda _s: MultivariateNormal(_s, std_dev_*torch.eye(_s.size()[-1]))
    rhos = list()
    for i in range(nb_simu):
        q2 = kernel_conv(q.float()).sample(torch.Size())
        rho = dist_Tp_Tq(p, q2.unsqueeze(0)).detach().numpy().item()
        rhos.append(rho)
    rho_final = np.mean(rhos)
    #std_rho = np.std(rhos)
    #print(f"Monte carlo phi: std = {std_rho}")
    return torch.tensor(rho_final)

In [ ]:
class NewPhi(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """
    def __init__(self):
        self.name = "New Phi class"

    @staticmethod
    def forward(ctx, input_q2, input_backward, p_torch, dist_Tp_Tq, std_dev_, nb_simu):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input_backward)
        monte_carlo_val = monte_carlo_phi(p_torch, input_q2, dist_Tp_Tq, std_dev_=std_dev_, nb_simu=nb_simu)
        return monte_carlo_val

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        back_,  = ctx.saved_tensors
        return back_ * grad_output, None, None, None, None, None

In [ ]:
s = torch.rand([24], requires_grad=True)
q = softplus(s)/torch.sum(softplus(s))
print(q)

kernel_conv = lambda _s: MultivariateNormal(_s, 1.0*torch.eye(_s.size()[-1]))

smooth_pg_2(dist_Tp_Tq, kernel_conv)(p_torch, q)

### Computing $\tilde q_{\varepsilon,\delta, p, T}$

Remember we want to compute
$$
\tilde q_{\varepsilon, p, T} = \arg\sup_{||p-q|| \leq \varepsilon} \phi(p,q)
$$
Practically, we solved the Lagrangian relaxation of this
$$
\tilde q_{\lambda, p, T} = \arg\sup_q \phi(p,q) - \lambda ||p-q||
$$

### WARNING

Here we use "smooth_pg_2", which approximates THE GRADIENT of phi, and then we call backward on that --> redondancy ??

In [ ]:
def torch_optim_attack_relaxed(dist_Tp_Tq, p_torch, reg, epochs, std_dev=0.01, lr=0.01):
    r"""
    :param method: $\tau_K(T(p), T(q))$
    :param reg: $\lambda$
    :param epochs: limit on the number of optimization iterations
    """
    p_torch2 = p_torch.detach().clone()
    p_torch2.requires_grad = False
    softplus = torch.nn.Softplus(beta=1, threshold=20)
    kernel_conv = lambda _s: MultivariateNormal(_s, std_dev*torch.eye(_s.size()[-1])) # $x\mapsto k_x(.)$
    smoothed_dist_Tp_Tq = lambda p, q: smooth_pg_2(dist_Tp_Tq, kernel_conv)(p, q)     # $\phi(.,.)$
    
    s_ = p_torch[0,:].detach().clone().to(device).type(default_tensor_type)
    s_.requires_grad = True
    optimizer = torch.optim.SGD([s_], lr=0.01, momentum=0.9)
    scheduler = CyclicLR(optimizer, 0.01, 1.0, step_size_down=50, cycle_momentum=False)

    for epoch in range(epochs):
        optimizer.zero_grad()
        q = softplus(s_)/torch.sum(softplus(s_))                                   # projection $\sum = 1$
        
        # Decrease the approximation error over time
        kernel_conv = lambda _s: MultivariateNormal(_s, std_dev*(10/(10+epoch))*torch.eye(_s.size()[-1])) # $x\mapsto k_x(.)$
        smoothed_dist_Tp_Tq = lambda p, q: smooth_pg_2(dist_Tp_Tq, kernel_conv)(p, q)     # $\phi(.,.)$
        
        loss = -smoothed_dist_Tp_Tq(p_torch2, q) + reg*(torch.norm(p_torch2-q, 1))     # Lagrangian relaxation of the objective
        loss.backward()
        optimizer.step()
        scheduler.step()
        

    return q

### Computing $
\tilde\varepsilon(\delta,p,T) = \inf \left\{\varepsilon>0 : \rho_{\tau_K}\left(T(p), T(\tilde q_{\varepsilon, p, T})\right) \geq \delta \right\}
$

First strategy: using $\rho_{\tau_K}$ in the evaluation of the breakdown point. Cons: piecewise-constant function, unstable results in the end.

Second strategy: using $\phi$ (the smoothed version of $\rho_{\tau_K}$). Cons: it seems the results are weird.

#### Monte-Carlo estimation of rho when using $\phi$

$\phi(p,q) = \rho_{\tau_K}(T(p), T(q)) \star k(p) = \int_u \rho_{\tau_K}(T(u), T(q)) \times k(p-u) du = \int_u \rho_{\tau_K}(T(u), T(q)) \times k_p(u) du = \mathbb{E}_{p' \sim k_p}(\rho_{\tau_K}(T(p'), T(q)))$ 

In [ ]:
# Get the optimal q
q = torch_optim_attack_relaxed(dist_Tp_Tq, p_torch, 10, epochs, std_dev=10, lr=0.01)

# Real result of rho(T(p), T(q))
print("Real result:", torch_dist_maxpair(p_torch, torch.unsqueeze(q,0), torch_all_ranks, threshold=threshold).detach().numpy())

# Defining phi
std_dev_ = 0.005
kernel_conv = lambda _s: MultivariateNormal(_s, std_dev_*torch.eye(_s.size()[-1])) # $x\mapsto k_x(.)$

# A large number of evaluations of phi
nb_simu = 10000
rhos = list()
for i in range(nb_simu):
    if i % 1000 == 0:
        print(f"simu nb {i}")
    p2 = kernel_conv(p_torch.float().squeeze(0)).sample(torch.Size())
    rho = dist_Tp_Tq(p2.unsqueeze(0), q.unsqueeze(0)).detach().numpy().item()
    rhos.append(rho)

# Mean approximation of phi
print(f"Mean = {np.mean(rhos)}")

# Plot
plt.hist(rhos, density=True, bins=25)
plt.show()

#### Implementation of the function to compute the breakdown point.

We want to solve the following pb:

$$\inf_{\varepsilon \geq 0} \varepsilon \text{ s.t. } \sup_{||p-q|| \leq \varepsilon} \rho_T(p,q) \geq \delta$$

This problem is equivalent to the following one, based one Lagrange relaxation:

$$ \inf_{\varepsilon \geq 0} \, \sup_{\lambda \geq 0} \, \inf_{q \in \Delta} \, \sup_{\alpha \geq 0} \, \, \varepsilon + \lambda \delta - \lambda \rho_T(p,q) + \alpha \lambda ||p-q|| - \lambda \alpha \varepsilon $$

To make it easy to follow, here is what each variable is controling:

1) $q$ is the attacking probability vector, which should not be too far from $p$, but change statistics $T$

2) $\varepsilon$ is the attack budget (the breakdown point) controling the perturbation from $q$

3) $\lambda$ controls that the statistics $T$ is indeed broken by at least $\delta$ by $q$

4) $\alpha$ controls that $q$ is indeed closer than $\varepsilon$ from $p$

In [ ]:
def subsample_fct(vect, time=10):
    l = len(vect)
    res = vect[0:l:time]
    return res

def approximate_breakdown_function(delta, dist_Tp_Tq, p_torch, epochs=150000, std_dev=0.01, lr=0.01, maxiter=10, max_reg=10., eval_strat="real"):
    if False:
        def _rho_minus_delta(_reg):
            q = torch_optim_attack_relaxed(dist_Tp_Tq, p_torch, _reg, epochs, std_dev=std_dev, lr=lr)
            if eval_strat == "real":
                rho = torch_dist_maxpair(p_torch, torch.unsqueeze(q,0), torch_all_ranks, threshold=threshold).detach().numpy()
            elif eval_strat == "smoothed":
                std_dev_ = 0.005
                nb_simu = 25000
                kernel_conv = lambda _s: MultivariateNormal(_s, std_dev_*torch.eye(_s.size()[-1])) # $x\mapsto k_x(.)$
                rhos = list()
                for i in range(nb_simu):
                    p2 = kernel_conv(p_torch.float().squeeze(0)).sample(torch.Size())
                    rho = dist_Tp_Tq(p2.unsqueeze(0), q.unsqueeze(0)).detach().numpy().item()
                    rhos.append(rho)
                rho = np.mean(rhos)
            else:
                print("Evaluation strategy not implemented")
            print(f"rho = {rho} and rho - delta = {rho - delta}")
            return rho - delta

        res = root_scalar(_rho_minus_delta, bracket=[1e-8, max_reg], x0=0.5, x1=2.5, xtol=None, rtol=None, maxiter=maxiter)
        q = torch_optim_attack_relaxed(dist_Tp_Tq, p_torch, res.root, epochs, std_dev=std_dev, lr=lr)
        return torch.norm(p_torch - torch.unsqueeze(q,0),1).detach().numpy()
    
    else:
        #q = torch_optim_attack_relaxed(dist_Tp_Tq, p_torch, epochs=20, std_dev=std_dev, reg=10, lr=0.01)
        q2 = p_torch.detach().clone().squeeze(0) #q.detach().clone() #
        q2.requires_grad = True
        s_ = torch.log(q2)[:].detach().clone().to(device).type(default_tensor_type)
        s_.requires_grad = True

        epsilon_ = torch.tensor(1.0, requires_grad=True)
        eta_ = torch.tensor(10.0, requires_grad=True)
        alpha_ = torch.tensor(10.0, requires_grad=True)
        qs_ = list()
        qs_total_ = list()
        epsilons_ = list()
        etas_ = list()
        alphas_ = list()

        mean_epsilons_ = list()
        mean_etas_ = list()
        mean_alphas_ = list()
        mean_qs_ = list()

        phis_ = list()
        losses = list()

        lr_list = list()

        optimizer_q2 = torch.optim.Adam([s_], lr=lr, betas=(0.9, 0.999), weight_decay=0.01)
        optimizer_eta = torch.optim.Adam([eta_], lr=lr, betas=(0.9, 0.999), weight_decay=0.01, maximize=True)
        optimizer_epsilon = torch.optim.Adam([epsilon_], lr=lr, betas=(0.9, 0.999), weight_decay=0.01)
        optimizer_alpha = torch.optim.Adam([alpha_], lr=lr, betas=(0.9, 0.999), weight_decay=0.01, maximize=True)


        scale_fct_ = lambda x: 0.5**x
        scheduler_q2 = torch.optim.lr_scheduler.CyclicLR(optimizer_q2, lr, 10, step_size_up=3000, mode='exp_range', gamma=0.8, cycle_momentum=False, scale_mode='cycle', scale_fn=scale_fct_)
        scheduler_eta = torch.optim.lr_scheduler.CyclicLR(optimizer_eta, lr, 10, step_size_up=3000, mode='exp_range', gamma=0.8, cycle_momentum=False, scale_mode='cycle', scale_fn=scale_fct_)
        scheduler_epsilon = torch.optim.lr_scheduler.CyclicLR(optimizer_epsilon, lr, 10, step_size_up=3000, mode='exp_range', gamma=0.8, cycle_momentum=False, scale_mode='cycle', scale_fn=scale_fct_)
        scheduler_alpha = torch.optim.lr_scheduler.CyclicLR(optimizer_alpha, lr, 10, step_size_up=3000, mode='exp_range', gamma=0.8, cycle_momentum=False, scale_mode='cycle', scale_fn=scale_fct_)

        for epoch in range(epochs):
            kernel_conv = lambda _s: MultivariateNormal(_s, 1/np.sqrt(1+epoch)*torch.eye(_s.size()[-1]))
            q2 = softplus(s_)/torch.sum(softplus(s_))
            res = smooth_pg_2(dist_Tp_Tq, kernel_conv)(p_torch, q2)
            res.backward(retain_graph=True)
            grad_data = s_.grad.data.detach().clone()
            Phi_ = NewPhi.apply
            phi_ = Phi_(q2, grad_data, p_torch, dist_Tp_Tq, 1/np.sqrt(1+epoch), 1)

            optimizer_q2.zero_grad()
            optimizer_eta.zero_grad()
            optimizer_epsilon.zero_grad()
            optimizer_alpha.zero_grad()

            loss = torch.exp(epsilon_) + softplus(eta_)/softplus(alpha_)*delta - softplus(eta_)/softplus(alpha_)*phi_ + softplus(eta_)*torch.norm(p_torch - torch.unsqueeze(q2,0),1) - softplus(eta_)*softplus(epsilon_)

            loss.backward()
            torch.nn.utils.clip_grad_value_(s_, 1)
            torch.nn.utils.clip_grad_value_(epsilon_, 1)
            torch.nn.utils.clip_grad_value_(eta_, 1)
            torch.nn.utils.clip_grad_value_(alpha_, 1)
            optimizer_q2.step()
            optimizer_eta.step()
            optimizer_epsilon.step()
            optimizer_alpha.step()

            scheduler_q2.step()
            scheduler_eta.step()
            scheduler_epsilon.step()
            scheduler_alpha.step()
            if epoch % 10 == 0:
                print(f"epoch {epoch}: \n \t epsilon_={softplus(epsilon_)} and grad = {-epsilon_.grad.data} and lr = {optimizer_epsilon.param_groups[0]['lr']} \n \t eta_={softplus(eta_)} and grad = {eta_.grad.data} and lr = {optimizer_eta.param_groups[0]['lr']} \n \t alpha_={softplus(alpha_)} and grad = {alpha_.grad.data} and lr = {optimizer_alpha.param_groups[0]['lr']} \n \t s_={s_}\n \t q2={q2} and sum={torch.sum(q2)} and grad = {-s_.grad.data} and lr = {optimizer_q2.param_groups[0]['lr']} \n \t phi = {phi_} and std_dev = {1.0/(1+epoch)}")
            losses.append(loss.detach().item())
            qs_total_.append(q2.detach().numpy())
            qs_.append(q2[0].detach().item())
            epsilons_.append(torch.exp(epsilon_).detach().item())
            etas_.append(torch.exp(eta_).detach().item())
            alphas_.append(torch.exp(alpha_).detach().item())
            phis_.append(phi_.item())
            lr_list.append(optimizer_epsilon.param_groups[0]['lr'])

            mean_qs_.append(np.mean(qs_total_[-50000:], axis=0))
            mean_epsilons_.append(np.mean(epsilons_[-50000:]))
            mean_etas_.append(np.mean(etas_[-50000:]))
            mean_alphas_.append(np.mean(alphas_[-50000:]))
            
            
        f, ax = plt.subplots(2,3, figsize=(15, 8))
        ax[0,0].plot(qs_)
        ax[0,0].set_title(f"First value of q")

        ax[0,1].plot(phis_)
        ax[0,1].set_title(f"Phi")

        ax[0,2].plot(epsilons_)
        ax[0,2].set_title(f"Epsilon")

        #ax[1,0].plot(lambdas_)
        #ax[1,0].set_title(f"Lambda")
        ax[1,0].plot(etas_)
        ax[1,0].set_title(f"Eta")

        ax[1,1].plot(alphas_)
        ax[1,1].set_title(f"Alpha")

        ax[1,2].plot(losses)
        ax[1,2].set_title(f"Loss")
        plt.show()
        
        print("\n \n")
        time_fct = 100
        x_axis_val = subsample_fct(np.linspace(0,len(mean_epsilons_),len(mean_epsilons_)), time=time_fct)
        f, ax = plt.subplots(2,3, figsize=(15, 8))

        mean_qs_subsample = subsample_fct(mean_qs_, time_fct)
        mean_qs1_ = [torch.norm(p_torch - torch.tensor(mean_q_), 1) for mean_q_ in mean_qs_subsample] #[mean_q_[0] for mean_q_ in mean_qs_]
        ax[0,0].plot(x_axis_val, mean_qs1_)
        ax[0,0].set_title(f"Norm of the difference between p and q")

        Phi_ = NewPhi.apply
        mean_phis_ = [Phi_(torch.tensor(mean_q_), grad_data, p_torch, dist_Tp_Tq, 0.1/(1+epoch), 10) for mean_q_ in mean_qs_subsample]
        ax[0,1].plot(x_axis_val, mean_phis_)
        ax[0,1].set_title(f"Phi")

        ax[0,2].plot(x_axis_val, subsample_fct(mean_epsilons_, time_fct))
        ax[0,2].set_title(f"Epsilon")

        #ax[1,0].plot(x_axis_val, subsample_fct(mean_lambdas_, time_fct))
        #ax[1,0].set_title(f"Lambda")
        ax[1,0].plot(x_axis_val, subsample_fct(mean_etas_, time_fct))
        ax[1,0].set_title(f"Eta")

        ax[1,1].plot(x_axis_val, subsample_fct(mean_alphas_, time_fct))
        ax[1,1].set_title(f"Alpha")

        ax[1,2].plot(x_axis_val, subsample_fct(losses, time_fct))
        ax[1,2].set_title(f"Loss")
        plt.plot()

        return losses, qs_, epsilons_, etas_, alphas_, s_, mean_qs_, mean_epsilons_, mean_etas_, mean_alphas_
    

### Warning :

You need to compile cells that follows this one (and the 3 following) first (I now, this does not make any sense). Please compile everything under "Family of distribution" and "Statistics under study" first.

In [ ]:
w = np.array([25, 8, 3, 2.5])
p = proba_plackett_luce(w, all_ranks)
p_torch = torch.from_numpy(p)
delta = 1
threshold = None
method_ = erm
softplus = torch.nn.Softplus(beta=1, threshold=20)
def torch_dist_maxpair(p_torch1, p_torch2, torch_all_ranks, threshold):
    R1 = method_(p_torch1, torch_all_ranks, n=4)#, threshold=threshold)
    R2 = method_(p_torch2, torch_all_ranks, n=4)#, threshold=threshold)
    return symmetrized_hausdorff_on_kendall(R1, R2)

method = torch_dist_maxpair
dist_Tp_Tq = lambda _p,_q: method(_p, _q, torch_all_ranks, threshold=threshold)

#q = torch_optim_attack_relaxed(dist_Tp_Tq, p_torch, 10, 20, std_dev=0.001, lr=0.01)
q2 = p_torch.detach().clone().squeeze(0) #q.detach().clone()
q2.requires_grad = True
s_ = torch.log(q2)[:].detach().clone().to(device).type(default_tensor_type)
print(s_.shape)
s_.requires_grad = True

epsilon_ = torch.tensor(2.0, requires_grad=True)
lambda_ = torch.tensor(2.0, requires_grad=True)
alpha_ = torch.tensor(3.0, requires_grad=True)
qs_ = list()
qs_total_ = list()
epsilons_ = list()
lambdas_ = list()
alphas_ = list()

mean_epsilons_ = list()
mean_lambdas_ = list()
mean_alphas_ = list()
mean_qs_ = list()

phis_ = list()
losses = list()

optimizer_q2 = torch.optim.Adam([s_], lr=10, betas=(0.9, 0.999), weight_decay=0.001)
optimizer_lambda = torch.optim.Adam([lambda_], lr=0.01, betas=(0.9, 0.999), weight_decay=0.001, maximize=True)
optimizer_epsilon = torch.optim.Adam([epsilon_], lr=0.1, betas=(0.9, 0.999), weight_decay=0.001)
optimizer_alpha = torch.optim.Adam([alpha_], lr=0.01, betas=(0.9, 0.999), weight_decay=0.001, maximize=True)

lambda_scheduler = lambda coef: lambda epoch: coef/np.sqrt(1+epoch) if epoch <= 5000 else 1/np.sqrt(1+5000)
scheduler_q2 = torch.optim.lr_scheduler.LambdaLR(optimizer_q2, lr_lambda=lambda_scheduler(1)) #.ReduceLROnPlateau(optimizer_q2, patience=10000, factor=0.75, verbose=True, mode="max")
scheduler_lambda = torch.optim.lr_scheduler.LambdaLR(optimizer_lambda, lr_lambda=lambda_scheduler(10)) #.ReduceLROnPlateau(optimizer_lambda, patience=10000, factor=0.75, verbose=True)
scheduler_epsilon = torch.optim.lr_scheduler.LambdaLR(optimizer_epsilon, lr_lambda=lambda_scheduler(1)) #.ReduceLROnPlateau(optimizer_epsilon, patience=10000, factor=0.75, verbose=True, mode="max")
scheduler_alpha = torch.optim.lr_scheduler.LambdaLR(optimizer_alpha, lr_lambda=lambda_scheduler(10)) #.ReduceLROnPlateau(optimizer_alpha, patience=10000, factor=0.75, verbose=True)

#scheduler_q2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_q2, patience=50, factor=0.95, verbose=True, mode="max")
#scheduler_lambda = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_lambda, patience=50, factor=0.85, verbose=True, mode="min")
#scheduler_epsilon = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_epsilon, patience=50, factor=0.95, verbose=True, mode="max")
#scheduler_alpha = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_alpha, patience=50, factor=0.5, verbose=True, mode="min")

for epoch in range(100):
    kernel_conv = lambda _s: MultivariateNormal(_s, 1/np.sqrt(1+epoch)*torch.eye(_s.size()[-1]))
    q2 = softplus(s_)/torch.sum(softplus(s_))
    res = smooth_pg_2(dist_Tp_Tq, kernel_conv)(p_torch, q2)
    res.backward(retain_graph=True)
    grad_data = s_.grad.data.detach().clone()
    Phi_ = NewPhi.apply
    phi_ = Phi_(q2, grad_data, p_torch, dist_Tp_Tq, 0.1/(1+epoch), 3)
    
    optimizer_q2.zero_grad()
    optimizer_lambda.zero_grad()
    optimizer_epsilon.zero_grad()
    optimizer_alpha.zero_grad()
    
    loss = torch.exp(epsilon_) + softplus(lambda_)*delta - softplus(lambda_)*phi_ + softplus(lambda_)*softplus(alpha_)*torch.norm(p_torch - torch.unsqueeze(q2,0),1) - softplus(lambda_)*softplus(alpha_)*softplus(epsilon_)
    
    loss.backward()
    optimizer_q2.step()
    optimizer_lambda.step()
    optimizer_epsilon.step()
    optimizer_alpha.step()
    
    scheduler_q2.step()#(loss.detach().item())
    scheduler_lambda.step()#(loss.detach().item())
    scheduler_epsilon.step()#(loss.detach().item())
    scheduler_alpha.step()#(loss.detach().item())
    if epoch % 10 == 0:
        print(f"epoch {epoch}: \n \t epsilon_={torch.exp(epsilon_)} and grad = {-epsilon_.grad.data} and lr = {optimizer_epsilon.param_groups[0]['lr']} \n \t lambda_={torch.exp(lambda_)} and grad = {lambda_.grad.data} and lr = {optimizer_lambda.param_groups[0]['lr']} \n \t alpha_={torch.exp(alpha_)} and grad = {alpha_.grad.data} and lr = {optimizer_alpha.param_groups[0]['lr']} \n \t s_={s_}\n \t q2={q2} and sum={torch.sum(q2)} and grad = {-s_.grad.data} and lr = {optimizer_q2.param_groups[0]['lr']} \n \t phi = {phi_} and std_dev = {1.0/(1+epoch)}")
    losses.append(loss.detach().item())
    qs_total_.append(q2.detach().numpy())
    qs_.append(q2[0].detach().item())
    epsilons_.append(torch.exp(epsilon_).detach().item())
    lambdas_.append(torch.exp(lambda_).detach().item())
    alphas_.append(torch.exp(alpha_).detach().item())
    phis_.append(phi_.item())
    
    mean_qs_.append(np.mean(qs_total_, axis=0))
    mean_epsilons_.append(np.mean(epsilons_))
    mean_lambdas_.append(np.mean(lambdas_))
    mean_alphas_.append(np.mean(alphas_))
    

In [ ]:
w = np.array([25, 8, 3, 2.5])
p = proba_plackett_luce(w, all_ranks)
p_torch = torch.from_numpy(p)
delta = 1
threshold = None
method_ = erm
softplus = torch.nn.Softplus(beta=1, threshold=20)
def torch_dist_maxpair(p_torch1, p_torch2, torch_all_ranks, threshold):
    R1 = method_(p_torch1, torch_all_ranks, n=4)#, threshold=threshold)
    R2 = method_(p_torch2, torch_all_ranks, n=4)#, threshold=threshold)
    return symmetrized_hausdorff_on_kendall(R1, R2)

method = torch_dist_maxpair
dist_Tp_Tq = lambda _p,_q: method(_p, _q, torch_all_ranks, threshold=threshold)

#q = torch_optim_attack_relaxed(dist_Tp_Tq, p_torch, 10, 20, std_dev=0.001, lr=0.01)
q2 = p_torch.detach().clone().squeeze(0) #q.detach().clone() #
q2.requires_grad = True
s_ = torch.log(q2)[:].detach().clone().to(device).type(default_tensor_type)
print(s_.shape)
s_.requires_grad = True

epsilon_ = torch.tensor(1.0, requires_grad=True)
eta_ = torch.tensor(10.0, requires_grad=True)
alpha_ = torch.tensor(10.0, requires_grad=True)
qs_ = list()
qs_total_ = list()
epsilons_ = list()
etas_ = list()
alphas_ = list()

mean_epsilons_ = list()
mean_etas_ = list()
mean_alphas_ = list()
mean_qs_ = list()

phis_ = list()
losses = list()

lr_list = list()

optimizer_q2 = torch.optim.Adam([s_], lr=0.01, betas=(0.9, 0.999), weight_decay=0.01)
optimizer_eta = torch.optim.Adam([eta_], lr=0.01, betas=(0.9, 0.999), weight_decay=0.01, maximize=True)
optimizer_epsilon = torch.optim.Adam([epsilon_], lr=0.01, betas=(0.9, 0.999), weight_decay=0.01)
optimizer_alpha = torch.optim.Adam([alpha_], lr=0.01, betas=(0.9, 0.999), weight_decay=0.01, maximize=True)


scale_fct_ = lambda x: 0.5**x
scheduler_q2 = torch.optim.lr_scheduler.CyclicLR(optimizer_q2, 0.01, 50, step_size_up=3000, mode='exp_range', gamma=0.8, cycle_momentum=False, scale_mode='cycle', scale_fn=scale_fct_)
scheduler_eta = torch.optim.lr_scheduler.CyclicLR(optimizer_eta, 0.01, 50, step_size_up=3000, mode='exp_range', gamma=0.8, cycle_momentum=False, scale_mode='cycle', scale_fn=scale_fct_)
scheduler_epsilon = torch.optim.lr_scheduler.CyclicLR(optimizer_epsilon, 0.01, 50, step_size_up=3000, mode='exp_range', gamma=0.8, cycle_momentum=False, scale_mode='cycle', scale_fn=scale_fct_)
scheduler_alpha = torch.optim.lr_scheduler.CyclicLR(optimizer_alpha, 0.01, 50, step_size_up=3000, mode='exp_range', gamma=0.8, cycle_momentum=False, scale_mode='cycle', scale_fn=scale_fct_)

#lambda_scheduler = lambda coef: lambda epoch: coef/np.sqrt(1+epoch) if epoch <= 25000 else 1/np.sqrt(1+25000)
#scheduler_q2 = torch.optim.lr_scheduler.LambdaLR(optimizer_q2, lr_lambda=lambda_scheduler(1)) #.ReduceLROnPlateau(optimizer_q2, patience=10000, factor=0.75, verbose=True, mode="max")
#scheduler_eta = torch.optim.lr_scheduler.LambdaLR(optimizer_eta, lr_lambda=lambda_scheduler(1)) #.ReduceLROnPlateau(optimizer_lambda, patience=10000, factor=0.75, verbose=True)
#scheduler_epsilon = torch.optim.lr_scheduler.LambdaLR(optimizer_epsilon, lr_lambda=lambda_scheduler(1)) #.ReduceLROnPlateau(optimizer_epsilon, patience=10000, factor=0.75, verbose=True, mode="max")
#scheduler_alpha = torch.optim.lr_scheduler.LambdaLR(optimizer_alpha, lr_lambda=lambda_scheduler(1)) #.ReduceLROnPlateau(optimizer_alpha, patience=10000, factor=0.75, verbose=True)

#scheduler_q2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_q2, patience=50, factor=0.95, verbose=True, mode="max")
#scheduler_lambda = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_lambda, patience=50, factor=0.85, verbose=True, mode="min")
#scheduler_epsilon = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_epsilon, patience=50, factor=0.95, verbose=True, mode="max")
#scheduler_alpha = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_alpha, patience=50, factor=0.5, verbose=True, mode="min")

for epoch in range(100):
    kernel_conv = lambda _s: MultivariateNormal(_s, 1/np.sqrt(1+epoch)*torch.eye(_s.size()[-1]))
    q2 = softplus(s_)/torch.sum(softplus(s_))
    res = smooth_pg_2(dist_Tp_Tq, kernel_conv)(p_torch, q2)
    res.backward(retain_graph=True)
    grad_data = s_.grad.data.detach().clone()
    Phi_ = NewPhi.apply
    phi_ = Phi_(q2, grad_data, p_torch, dist_Tp_Tq, 1/np.sqrt(1+epoch), 1)
    
    optimizer_q2.zero_grad()
    optimizer_eta.zero_grad()
    optimizer_epsilon.zero_grad()
    optimizer_alpha.zero_grad()
    
    loss = torch.exp(epsilon_) + softplus(eta_)/softplus(alpha_)*delta - softplus(eta_)/softplus(alpha_)*phi_ + softplus(eta_)*torch.norm(p_torch - torch.unsqueeze(q2,0),1) - softplus(eta_)*softplus(epsilon_)
    
    loss.backward()
    torch.nn.utils.clip_grad_value_(s_, 1)
    torch.nn.utils.clip_grad_value_(epsilon_, 1)
    torch.nn.utils.clip_grad_value_(eta_, 1)
    torch.nn.utils.clip_grad_value_(alpha_, 1)
    optimizer_q2.step()
    optimizer_eta.step()
    optimizer_epsilon.step()
    optimizer_alpha.step()
    
    scheduler_q2.step()#(loss.detach().item())
    scheduler_eta.step()#(loss.detach().item())
    scheduler_epsilon.step()#(loss.detach().item())
    scheduler_alpha.step()#(loss.detach().item())
    if epoch % 10 == 0:
        print(f"epoch {epoch}: \n \t epsilon_={softplus(epsilon_)} and grad = {-epsilon_.grad.data} and lr = {optimizer_epsilon.param_groups[0]['lr']} \n \t eta_={softplus(eta_)} and grad = {eta_.grad.data} and lr = {optimizer_eta.param_groups[0]['lr']} \n \t alpha_={softplus(alpha_)} and grad = {alpha_.grad.data} and lr = {optimizer_alpha.param_groups[0]['lr']} \n \t s_={s_}\n \t q2={q2} and sum={torch.sum(q2)} and grad = {-s_.grad.data} and lr = {optimizer_q2.param_groups[0]['lr']} \n \t phi = {phi_} and std_dev = {1.0/(1+epoch)}")
    losses.append(loss.detach().item())
    qs_total_.append(q2.detach().numpy())
    qs_.append(q2[0].detach().item())
    epsilons_.append(torch.exp(epsilon_).detach().item())
    etas_.append(torch.exp(eta_).detach().item())
    alphas_.append(torch.exp(alpha_).detach().item())
    phis_.append(phi_.item())
    lr_list.append(optimizer_epsilon.param_groups[0]['lr'])
    
    mean_qs_.append(np.mean(qs_total_[-50000:], axis=0))
    mean_epsilons_.append(np.mean(epsilons_[-50000:]))
    mean_etas_.append(np.mean(etas_[-50000:]))
    mean_alphas_.append(np.mean(alphas_[-50000:]))

In [ ]:
print(epsilons_[-1])
print(etas_[-1])
#print(lambdas_[-1])
print(alphas_[-1])
print(qs_[-1])

plt.plot(lr_list)
plt.show()
lr_list[-1]

In [ ]:
f, ax = plt.subplots(2,3, figsize=(15, 8))

l_ = 10000

ax[0,0].plot(qs_[-l_:])
ax[0,0].set_title(f"First value of q")

ax[0,1].plot(phis_[-l_:])
ax[0,1].set_title(f"Phi")

ax[0,2].plot(epsilons_[-l_:])
ax[0,2].set_title(f"Epsilon")

#ax[1,0].plot(lambdas_)
#ax[1,0].set_title(f"Lambda")
ax[1,0].plot(etas_[-l_:])
ax[1,0].set_title(f"Eta")

ax[1,1].plot(alphas_[-l_:])
ax[1,1].set_title(f"Alpha")

ax[1,2].plot(losses[-l_:])
ax[1,2].set_title(f"Loss")


In [ ]:
def subsample_fct(vect, time=10):
    l = len(vect)
    res = vect[0:l:time]
    return res
time_fct = 10

x_axis_val = subsample_fct(np.linspace(0,len(mean_epsilons_),len(mean_epsilons_)), time=time_fct)

f, ax = plt.subplots(2,3, figsize=(15, 8))

mean_qs_subsample = subsample_fct(mean_qs_, time_fct)
mean_qs1_ = [torch.norm(p_torch - torch.tensor(mean_q_), 1) for mean_q_ in mean_qs_subsample] #[mean_q_[0] for mean_q_ in mean_qs_]
ax[0,0].plot(x_axis_val, mean_qs1_)
ax[0,0].set_title(f"Norm of the difference between p and q")

Phi_ = NewPhi.apply
mean_phis_ = [Phi_(torch.tensor(mean_q_), grad_data, p_torch, dist_Tp_Tq, 0.1/(1+epoch), 10) for mean_q_ in mean_qs_subsample]
ax[0,1].plot(x_axis_val, mean_phis_)
ax[0,1].set_title(f"Phi")

ax[0,2].plot(x_axis_val, subsample_fct(mean_epsilons_, time_fct))
ax[0,2].set_title(f"Epsilon")

#ax[1,0].plot(x_axis_val, subsample_fct(mean_lambdas_, time_fct))
#ax[1,0].set_title(f"Lambda")
ax[1,0].plot(x_axis_val, subsample_fct(mean_etas_, time_fct))
ax[1,0].set_title(f"Eta")

ax[1,1].plot(x_axis_val, subsample_fct(mean_alphas_, time_fct))
ax[1,1].set_title(f"Alpha")

ax[1,2].plot(x_axis_val, subsample_fct(losses, time_fct))
ax[1,2].set_title(f"Loss")


In [ ]:
print(mean_qs_subsample[-1])
print(mean_epsilons_[-1])
print(Phi_(torch.tensor(mean_qs_subsample[-1]), grad_data, p_torch, dist_Tp_Tq, 0.001, 100))
print(erm(torch.tensor(mean_qs_subsample[-1]).unsqueeze(0), torch_all_ranks, n=4))

In [ ]:
print(p_torch)
print(q2)
print(torch.norm(p_torch.squeeze(0)-q2, 1))
print(erm(q2.unsqueeze(0), torch_all_ranks, n=4))

In [ ]:
phi_smooth = phis_[-1] #monte_carlo_phi(p_torch, q2, dist_Tp_Tq, nb_simu=100, std_dev_=0.0001).item()
myres = delta - np.asarray(alphas_)*np.asarray(epsilons_) + np.asarray(alphas_)*torch.norm(p_torch.squeeze(0)-q2, 1).item() - phi_smooth

plt.plot(myres)
plt.show()

print(myres[-1])
print((np.asarray(alphas_)*np.asarray(epsilons_))[-1])
print((np.asarray(alphas_)*torch.norm(p_torch.squeeze(0)-q2, 1).item())[-1])
print(f"phi = {dist_Tp_Tq(p_torch, q2.unsqueeze(0))} vs phi_smooth = {phi_smooth}")
print(q2)


# Family of distribution

The Plackett-Luce model is a parametric family of distributions over permutations:
$$
\mathbb{P}\left(\sigma | w\right) = \prod_{r=1}^n \frac{w_{\sigma(r)}}{\sum_{k\geq r}w_{\sigma(k)}}
$$

In [ ]:
def proba_plackett_luce(w, all_ranks, n_items=4):
    list_proba = list()

    for sigma in all_ranks:
        val_ = list()
        for r in range(n_items):
            num_ = w[sigma[r]]
            denom_ = 0
            for s in range(r, n_items):
                v_ = w[sigma[s]]
                denom_ += v_
            val_.append(num_/denom_)
        proba_ = np.prod(val_)
        list_proba.append(proba_)
    return np.matrix(list_proba)

# Statistics under study

### MaxPair

Todo: add a description

In [ ]:
def _maxpair(P, threshold = 0.):
    s = torch.sum(1*(P > 0.5)+1/2*(P == 0.5), axis=1)
    sigma = torch.argsort(-s)
    M = P[np.ix_(sigma,sigma)]
    
    idxs = torch.argwhere(M>1/2+threshold)
    for idx in idxs:
        M[:idx[0]+1,idx[1]] = 1
        M[idx[1],:idx[0]+1] = 0

    m = torch.max(torch.abs(M-0.5)*(M!=0.5)*(torch.abs(M-0.5) <= threshold))
    while m > 0:
        i,j = torch.argwhere(np.abs(M-0.5) == m)[0,0], torch.argwhere(np.abs(M-0.5) == m)[0,1]
        if i <= j:
            idx_tomerge1, idx_tomerge2 = i, j+1
        elif i > j:
            idx_tomerge1, idx_tomerge2 = j, i+1
        M = torch_merge_idx(M, torch.arange(idx_tomerge1,idx_tomerge2))
        m = torch.max(np.abs(M - 0.5) * (M != 0.5) * (torch.abs(M - 0.5) <= threshold))
    return M[np.ix_(torch.argsort(sigma), torch.argsort(sigma))]

def maxpair(p, torch_all_ranks, n=4, threshold = 0.):
    P = pairwise_matrix(p, torch_all_ranks=torch_all_ranks, n=n)
    return _maxpair(P, threshold = threshold)

### Merge

Todo: add a description

In [ ]:
def torch_merge_idx(M, idx):
    # i,j \notin idx -> P_ij = M_ij
    # i \in idx, j \notin idx -> P_ij = \max_{k\in idx} M_kj
    # i \notin idx, j \in idx -> P_ij = \max_{k\in idx} M_ik
    # i,j \in idx -> P_ij = 0.5
    P = M
    for i in torch.arange(M.shape[0]):
        m = torch.max(M[i, idx])
        for j in idx:
            P[i,j] = m
    for j in torch.arange(M.shape[0]):
        m = torch.max(M[idx, j])
        for i in idx:
            P[i,j] = m
    for i in idx:
        for j in idx:
            P[i,j] = 0.5
    PTRIU = torch.triu(P, 0)
    P = PTRIU + torch.tril(1 - PTRIU.T, -1)
    return P

def merge(p, torch_all_ranks, threshold=0, n=4):
    P = pairwise_matrix(p, torch_all_ranks=torch_all_ranks, n=n)
    cont = True
    while cont:
        P_mod = torch.abs(torch.triu(P,1)-torch.tensor(0.5))
        m = torch.min(torch.abs(torch.triu(P,1)-torch.tensor(0.5)))
        if m == 0.0 or m > threshold:
            cont = False
        else:
            idxs = torch.argwhere(P_mod == m)
            idxs = idxs.reshape(torch.prod(torch.tensor(idxs.shape)))
            P = torch_merge_idx(P, idxs)
    return P

### ERM

Todo: add a description

In [ ]:
def erm(p, torch_all_ranks, n=4):
    P = pairwise_matrix(p, torch_all_ranks=torch_all_ranks, n=n)
    return torch.round(P)

### Depth

Todo: add a description

Only available for n=4 items

In [ ]:
def kendall_tau(sigma1, sigma2):
    n = sigma1.size()[-1]
    sigma1_inv = torch.argsort(sigma1, dim=-1)
    sigma2_inv = torch.argsort(sigma2, dim=-1)
    sigma1_pairs = (sigma1_inv.unsqueeze(dim=-1) > sigma1_inv.unsqueeze(dim=-2)).float()
    sigma2_pairs = (sigma2_inv.unsqueeze(dim=-1) > sigma2_inv.unsqueeze(dim=-2)).float()
    return torch.sum(torch.abs(sigma1_pairs-sigma2_pairs), dim=[-2,-1]).double()/2 #/(n*(n-1))

def kendall_matrix(torch_all_ranks):
    K = torch.zeros(len(torch_all_ranks),len(torch_all_ranks))
    for i, rank1 in enumerate(torch_all_ranks):
        for j, rank2 in enumerate(torch_all_ranks):
            K[i,j] = kendall_tau(rank1,rank2)
    return K.double()

def get_all_buckets(torch_all_ranks, n=4):
    list_bucket = list()

    for rank1 in torch_all_ranks:
        for rank2 in torch_all_ranks:
            if kendall_tau(rank1,rank2) == 1.0:
                list_bucket.append( [rank1,rank2] )
    
    for i in np.arange(n):
        temp_ranks = list()
        for rank in torch_all_ranks:
            if rank[3] == i:
                temp_ranks.append(rank)
        list_bucket.append(temp_ranks)
        
    list_bucket.append([torch.tensor([0,1,2,3]), torch.tensor([0,1,3,2]),torch.tensor([1,0,2,3]),torch.tensor([1,0,3,2])])
    list_bucket.append([torch.tensor([0,2,1,3]), torch.tensor([0,2,3,1]),torch.tensor([2,0,1,3]),torch.tensor([2,0,3,1])])
    list_bucket.append([torch.tensor([0,3,1,2]), torch.tensor([0,3,2,1]),torch.tensor([3,0,1,2]),torch.tensor([3,0,2,1])])
    list_bucket.append([torch.tensor([1,2,0,3]), torch.tensor([1,2,3,0]),torch.tensor([2,1,0,3]),torch.tensor([2,1,3,0])]) 
    list_bucket.append([torch.tensor([1,3,0,2]), torch.tensor([1,3,2,0]),torch.tensor([3,1,0,2]),torch.tensor([3,1,2,0])])
    list_bucket.append([torch.tensor([2,3,0,1]), torch.tensor([2,3,1,0]),torch.tensor([3,2,0,1]),torch.tensor([3,2,1,0])])

    list_bucket.append(torch_all_ranks)
    for rank in torch_all_ranks:
        list_bucket.append(list([rank]))
    
    return list_bucket

def bucket_distrib(torch_all_ranks, list_bucket):
    l_ = list()
    div = len(list_bucket)
    for a_ in torch_all_ranks:
        count = 0
        for b_ in list_bucket:
            count += 1
            if (b_ == a_).all():
                l_.append(1.0/div)
                count += 1
                continue
        if count == div:
            l_.append(0)
    return torch.tensor(l_).double()

def depth_metric_optim(p, K, list_bucket, torch_all_ranks, norm="l1", printer=False):
    n_ranks_to_test = len(list_bucket)
    val = torch.inf
    for i, bucket in enumerate(list_bucket):
        q = bucket_distrib(torch_all_ranks, bucket)
        if norm == "l1":
            val_ = torch.norm(torch.matmul(p.double(),K) - torch.matmul(q.double(),K), 1)
        elif norm == "l2":
            val_ = torch.norm(torch.matmul(p.double(),K) - torch.matmul(q.double(),K), 2)**2
        if printer:
            print(f"val: {val_} for {bucket}")
        if val_ <= val:
            best_distrib = q
            val = val_
    return best_distrib, val

def depth(p, torch_all_ranks, norm="l1", printer=False):
    list_bucket = get_all_buckets(torch_all_ranks, n=4)
    K = kendall_matrix(torch_all_ranks)
    q, val = depth_metric_optim(p, K, list_bucket, torch_all_ranks, norm=norm, printer=printer)
    Q = pairwise_matrix(q.unsqueeze(0), torch_all_ranks=torch_all_ranks, n=4)
    return Q  
    

### Wasserstein

Todo: add a description

In [ ]:
from scipy import optimize
from math import factorial

def wasserstein_dist(p, q, K, n=4):
    nsize = factorial(n)
    K_bis = K.view(nsize*nsize)
    
    for i in range(nsize):
        A_ = [0]*nsize*i + [1]*nsize + [0]*nsize*(nsize-i-1)
        if i == 0:
            A = torch.tensor(A_).reshape(1,nsize*nsize)
        else:
            A = torch.cat((A, torch.tensor(A_).reshape(1,nsize*nsize)))
    for i in range(nsize):
        A_ = ([0]*i + [1] + [0]*(nsize-i-1))*nsize
        A = torch.cat((A, torch.tensor(A_).reshape(1,nsize*nsize)))
    
    b = torch.cat((p, q), 1)
    
    optim_val = optimize.linprog(K_bis, A_eq=A, b_eq=b, bounds=(0,1))
    return optim_val.fun

def wasserstein_(myp, torch_all_ranks, n=4, printer=False):
    list_bucket = get_all_buckets(torch_all_ranks, n=4)
    K = kendall_matrix(torch_all_ranks)
    val = torch.inf
    for i, bucket in enumerate(list_bucket):
        q = bucket_distrib(torch_all_ranks, bucket)
        if len(q.shape) < 2:
            q = q.unsqueeze(0)
        val_ = wasserstein_dist(myp, q, K, n=n)
        #if printer:
        print(f"WASS i = {i} --> val: {val_} for {bucket} / q = {q} and p = {myp}")
        if val_ <= val:
            best_distrib = q
            val = val_
    print(f"END WASSERSTEIN_ with {val_} and {best_distrib}")
    return best_distrib, val

def wasserstein(myp, torch_all_ranks, n=4):
    print(f"CALL TO WASSERSTEIN")
    q, val = wasserstein_(myp, torch_all_ranks, n=n, printer=False)
    Q = pairwise_matrix(q, torch_all_ranks=torch_all_ranks, n=n)
    print(f"END OF CALL TO WASSERSTEIN")
    return Q

# Simulations

## Checking the converge of the optimization of the attack

In [ ]:
w = np.array([25, 8, 3, 2.5])
p = proba_plackett_luce(w, all_ranks)
p_torch = torch.from_numpy(p)

torch_all_ranks = torch.from_numpy(np.asarray(all_ranks))
P = pairwise_matrix(p_torch, torch_all_ranks, n=4)

method_ = maxpair #depth
def torch_dist_maxpair(p_torch1, p_torch2, torch_all_ranks, threshold):
    R1 = method_(p_torch1, torch_all_ranks, n=4, threshold=threshold)
    R2 = method_(p_torch2, torch_all_ranks, n=4, threshold=threshold)
    return symmetrized_hausdorff_on_kendall(R1, R2)

method = torch_dist_maxpair

epochs = 1000
threshold = 0.0

dist_Tp_Tq = lambda _p,_q: method(_p, _q, torch_all_ranks, threshold=threshold)

softplus = torch.nn.Softplus(beta=1, threshold=20)
kernel_conv = lambda _s: MultivariateNormal(_s, 0.1*torch.eye(_s.size()[-1])) # $x\mapsto k_x(.)$
smoothed_dist_Tp_Tq = lambda p, q: smooth_pg_2(dist_Tp_Tq, kernel_conv)(p, q)     # $\phi(.,.)$

s_ = p_torch[0,:].detach().clone().to(device).type(default_tensor_type)
s_.requires_grad = True
optimizer = torch.optim.SGD([s_], lr=0.01, momentum=0.9)
scheduler = CyclicLR(optimizer, 0.01, 1.0, step_size_down=50, cycle_momentum=False)

losses = []
noreglosses = []
metrics = []
for epoch in range(epochs):
    optimizer.zero_grad()
    q = softplus(s_)/torch.sum(softplus(s_))                                      # projection $\sum = 1$
    
    kernel_conv = lambda _s: MultivariateNormal(_s, (10/(10+epoch))*torch.eye(_s.size()[-1])) # $x\mapsto k_x(.)$
    smoothed_dist_Tp_Tq = lambda p, q: smooth_pg_2(dist_Tp_Tq, kernel_conv)(p, q)     # $\phi(.,.)$

    loss = -smoothed_dist_Tp_Tq(p_torch, q) + 2.5*(torch.norm(p_torch-q, 1))      # Lagrangian relaxation of the objective
    loss.backward()
    optimizer.step()
    scheduler.step()
    losses.append(loss.detach().numpy())
    noreglosses.append(smoothed_dist_Tp_Tq(p_torch, q).detach().numpy())
    metrics.append(dist_Tp_Tq(p_torch, torch.unsqueeze(q,0)).detach().numpy())

In [ ]:
def movingaverage(interval, window_size=10):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')
    
plt.plot(movingaverage(np.ravel(losses)/100), 'b')
plt.plot(movingaverage(-np.ravel(noreglosses)/100), 'r')
plt.plot(np.ravel(metrics), 'g')
#plt.yscale('log')

After some tests, it seems using a plain SGD with cyclic stepsize is a good way to handle the specific landscape of the objective that alternates between plateau and sharp slopes.

**However**, the optimization does not seems satisfactory yet, as highlighted by the following experiments that shows the estimate of $\tilde \varepsilon$ as a function of $\delta$ is not monotonous.

**Hypothesis**: As we are looking for the minimal budget $\varepsilon^*$ that allows to break the statistic $T$ by at least $\delta$, it means that, for an $\varepsilon$ close to $\varepsilon^*$, most of the feasible attacks $\tilde q$ break the statistic $T$ by **strictly less** than $\delta$. At the limit of $\varepsilon \to \varepsilon^*_+$, only the optimal attack breaks the statistic by $\delta$, which explains that even assymptotically during the optimization (see figure above), the value of $\delta$ is not fully stable.

## Studying the computation of $
\tilde\varepsilon(\delta,p,T)$

In [ ]:
w = np.array([25, 8, 3, 2.5])
p = proba_plackett_luce(w, all_ranks)
p_torch = torch.from_numpy(p)

torch_all_ranks = torch.from_numpy(np.asarray(all_ranks))
P = pairwise_matrix(p_torch, torch_all_ranks, n=4)
dist = "erm"

def torch_dist(dist, p_torch1, p_torch2, torch_all_ranks, threshold, dist_type_sym=True):
    if dist == "erm":
        R1 = erm(p_torch1, torch_all_ranks, n=4)
        R2 = erm(p_torch2, torch_all_ranks, n=4)
    elif dist == "maxpair":
        R1 = maxpair(p_torch1, torch_all_ranks, n=4, threshold=threshold)
        R2 = maxpair(p_torch2, torch_all_ranks, n=4, threshold=threshold)
    elif dist == "merge":
        R1 = merge(p_torch1, torch_all_ranks, threshold=threshold, n=4)
        R2 = merge(p_torch2, torch_all_ranks, threshold=threshold, n=4)
    elif dist == "depth":
        R1 = depth(p_torch1, torch_all_ranks, norm="l1", printer=False)
        R2 = depth(p_torch2, torch_all_ranks, norm="l1", printer=False)
    elif dist == "wasserstein":
        R1 = wasserstein(p_torch1, torch_all_ranks, n=4)
        R2 = wasserstein(p_torch2, torch_all_ranks, n=4)
    if dist_type_sym:
        return symmetrized_hausdorff_on_kendall(R1, R2)
    else:
        return disymmetrized_hausdorff_on_kendall(R1, R2)

method = torch_dist

epochs = 4000
threshold = 0.0

dist_Tp_Tq = lambda _p,_q: method(dist, _p, _q, torch_all_ranks, threshold=threshold)

delta = 1.0
losses, epsilons, q2 = approximate_breakdown_function(delta, dist_Tp_Tq, p_torch, epochs=epochs, std_dev=0.1, lr=0.01, max_reg=10, maxiter=11, eval_strat="smoothed")
#eps = losses[-1]

plt.plot(losses)
plt.show()

print("EPS(DELTA="+str(delta)+"):", epsilons[-1])

#regs = np.linspace(0., 4, 21)
#q_list = [torch_optim_attack_relaxed(dist_Tp_Tq, p_torch, reg, epochs, std_dev=10, lr=0.01) for reg in regs]

In [ ]:
print(p_torch)
print(q2)
print(torch.norm(p_torch - q2, 1))

erm(q2.unsqueeze(0), torch_all_ranks, n=4)

In [ ]:
eps_list = [torch.norm(p_torch - torch.unsqueeze(_q,0),1).detach().numpy() for _q in q_list]
delta_list = [dist_Tp_Tq(p_torch, torch.unsqueeze(_q,0)) for _q in q_list]

**SANITY CHECK:** The attack budget $\varepsilon$ is a (decreasing) function of the Lagrange parameter $\lambda$.

In [ ]:
plt.plot(regs, eps_list)
plt.title(f"Attack budget (eps) as a function of regularization")
plt.xlabel(f"Regularization")
plt.ylabel(f"Attack budget (eps)")
plt.show()

**SANITY CHECK:** The attack is stronger for lower Lagrange variable values.

In [ ]:
plt.plot(regs, delta_list)
plt.title(f"Loss as a function of regularization")
plt.xlabel(f"Regularization")
plt.ylabel(f"Loss")
plt.show()

**RESULT:** $\varepsilon^*(\delta,p,T)$ (lower is better)

In [ ]:
plt.plot(delta_list, eps_list)
plt.title(f"Loss as a function of attack budget (eps)")
plt.xlabel(f"Attack budget (eps)")
plt.ylabel(f"Loss")
plt.show()

## Performance vs Robustness profile

**WARNING** The following cell takes a lot of time. It's still not finalized as the computation of $\tilde\varepsilon$ is not yet reliable.

In [ ]:
dist = "erm"
w = np.array([25, 8, 3, 2.5])
delta = 1
epochs = 100 #150000 #True val is 150000
thresholds_ = np.linspace(0., 0.5, 3)

def launch_exp(dist, w, delta, thresholds_, epochs):
    p = proba_plackett_luce(w, all_ranks)
    p_torch = torch.from_numpy(p)
    torch_all_ranks = torch.from_numpy(np.asarray(all_ranks))
    P = pairwise_matrix(p_torch, torch_all_ranks, n=4)

    def torch_dist(dist, p_torch1, p_torch2, torch_all_ranks, threshold, dist_type_sym=True):
        if dist == "erm":
            R1 = erm(p_torch1, torch_all_ranks, n=4)
            R2 = erm(p_torch2, torch_all_ranks, n=4)
        elif dist == "maxpair":
            R1 = maxpair(p_torch1, torch_all_ranks, n=4, threshold=threshold)
            R2 = maxpair(p_torch2, torch_all_ranks, n=4, threshold=threshold)
        elif dist == "merge":
            R1 = merge(p_torch1, torch_all_ranks, threshold=threshold, n=4)
            R2 = merge(p_torch2, torch_all_ranks, threshold=threshold, n=4)
        elif dist == "depth":
            R1 = depth(p_torch1, torch_all_ranks, norm="l1", printer=False)
            R2 = depth(p_torch2, torch_all_ranks, norm="l1", printer=False)
        elif dist == "wasserstein":
            R1 = wasserstein(p_torch1, torch_all_ranks, n=4)
            R2 = wasserstein(p_torch2, torch_all_ranks, n=4)
        if dist_type_sym:
            return symmetrized_hausdorff_on_kendall(R1, R2)
        else:
            return disymmetrized_hausdorff_on_kendall(R1, R2)

    eps_list1 = []
    eps_list2 = []
    perf_list = []
    if dist in ["erm", "depth", "wasserstein"]:
        thresholds = [0]
    elif dist in ["maxpair", "merge"]:
        thresholds = thresholds_
        
    for threshold in thresholds:
        print(f"\n \t EXP THRESHOLD {threshold} \n \n \n")
        dist_Tp_Tq = lambda _p,_q: torch_dist(dist, _p, _q, torch_all_ranks, threshold=threshold)
        losses, qs_, epsilons, etas, alphas, s_, mean_qs, mean_epsilons, mean_etas, mean_alphas = approximate_breakdown_function(delta-1e-6, dist_Tp_Tq, p_torch, epochs=epochs, std_dev=0.0001, lr=0.01, maxiter=21, eval_strat="smoothed")
        q2 = mean_qs[-1]
        eps_list1.append(mean_epsilons[-1])
        eps_list2.append(torch.norm(p_torch - torch.tensor(q2).unsqueeze(0), 1))
        if dist == "erm":
            Ptilde = erm(p_torch, torch_all_ranks, n=4)
        elif dist == "maxpair":
            Ptilde = maxpair(p_torch, torch_all_ranks, n=4, threshold=threshold)
        elif dist == "merge":
            Ptilde = merge(p_torch, torch_all_ranks, threshold=threshold, n=4)
        elif dist == "depth":
            Ptilde = depth(p_torch, torch_all_ranks, norm="l1", printer=False)
        elif dist == "wasserstein":
            Ptilde = wasserstein(p_torch, torch_all_ranks, n=4)
        exp_kendall = expected_kendall(Ptilde, P).detach().item()
        perf_list.append(exp_kendall)
    
    return perf_list, eps_list1, eps_list2

In [ ]:
# ERM
dist = "erm"
perf_list_erm, eps_list_erm1, eps_list_erm2 = launch_exp(dist, w, delta, thresholds_, epochs)

In [ ]:
print(f"perf_list = {perf_list_erm} and eps_list1 = {eps_list_erm1} and eps_list2 = {eps_list_erm2}")

plt.plot(perf_list_erm, eps_list_erm1, '*')
plt.plot(perf_list_erm, eps_list_erm2, '*')
plt.show()

In [ ]:
# Depth
dist = "depth"
perf_list_depth, eps_list_depth1, eps_list_depth2 = launch_exp(dist, w, delta, thresholds_, epochs)

In [ ]:
print(f"perf_list = {perf_list_depth} and eps_list = {eps_list_depth}")
plt.plot(perf_list_depth, eps_list_depth1, '*')
plt.plot(perf_list_depth, eps_list_depth2, '*')
plt.show()

In [ ]:
# Maxpair
dist = "maxpair"
perf_list_maxpair, eps_list_maxpair1, eps_list_maxpair2 = launch_exp(dist, w, delta, thresholds_, epochs)

In [ ]:
print(f"perf_list = {perf_list_maxpair} and eps_list1 = {eps_list_maxpair1} and eps_list2 = {eps_list_maxpair2}")
plt.plot(perf_list_maxpair, eps_list_maxpair1, '*')
plt.plot(perf_list_maxpair, eps_list_maxpair2, '*')
plt.show()

In [ ]:
# Merge
dist = "merge"
perf_list_merge, eps_list_merge1, eps_list_merge2 = launch_exp(dist, w, delta, thresholds_, epochs)

In [ ]:
print(f"perf_list = {perf_list_merge} and eps_list = {eps_list_merge}")
plt.plot(perf_list_merge, eps_list_merge1, '*')
plt.plot(perf_list_merge, eps_list_merge2, '*')
plt.show()

In [ ]:
# Wasserstein
dist = "wasserstein"
perf_list_wasserstein, eps_list_wasserstein1, eps_list_wasserstein2 = launch_exp(dist, w, delta, thresholds_, epochs)

In [ ]:
print(f"perf_list = {perf_list_wasserstein} and eps_list = {eps_list_wasserstein}")
plt.plot(perf_list_wasserstein, eps_list_wasserstein1, '*')
plt.plot(perf_list_wasserstein, eps_list_wasserstein2, '*')
plt.show()

In [ ]:
K = kendall_matrix(torch_all_ranks)
p = torch.tensor([[0.0369, 0.0388, 0.0497, 0.0311, 0.0455, 0.0496, 0.0381, 0.0308, 0.0266,
         0.0475, 0.0370, 0.0599, 0.0317, 0.0472, 0.0301, 0.0499, 0.0358, 0.0273,
         0.0548, 0.0422, 0.0442, 0.0240, 0.0388, 0.0448]])
print(torch.sum(p))
q = torch.tensor([[0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]], dtype=torch.float64)
val_ = wasserstein_dist(p, q, K, n=4)

In [ ]:
q

In [ ]:
plt.plot(perf_list_erm, eps_list_erm, 'x')
plt.plot(perf_list_merge, eps_list_merge, 'o')
plt.plot(perf_list_maxpair, eps_list_maxpair, '*')
plt.plot(perf_list_depth, eps_list_depth, 's')
plt.xlabel("Performance (loss)")
plt.ylabel("Robustness (bkdwn pts)")
plt.show()

# UNUSED CODE

In [ ]:
def kendall_tau_numpy(y, sigma):
    y = torch.from_numpy(y)
    sigma = torch.from_numpy(sigma)
    n = sigma.size()[-1]
    sigma_inv = torch.argsort(sigma, dim=-1)
    y_inv = torch.argsort(y, dim=-1)
    sigma_pairs = (sigma_inv.unsqueeze(dim=-1) > sigma_inv.unsqueeze(dim=-2)).float()
    y_pairs = (y_inv.unsqueeze(dim=-1) > y_inv.unsqueeze(dim=-2)).float()
    return torch.sum(torch.abs(sigma_pairs-y_pairs), dim=[-2,-1])/2 #/(n*(n-1))

def kendall_matrix(all_ranks):
    K = np.zeros((len(all_ranks),len(all_ranks)))
    for i, rank1 in enumerate(all_ranks):
        for j, rank2 in enumerate(all_ranks):
            K[i,j] = kendall_tau_numpy(rank1,rank2)
    return K

def kendall_tau(y, sigma, norm=False):
    """
    Kendall Tau distance
    Permutations follow the convention rank -> item_id
    :param torch.Tensor y: permutation
    :param torch.Tensor sigma: permutation
    :return: value of the metric
    :rtype: torch.Tensor
    """
    with torch.no_grad():
        n = sigma.size()[-1]
        sigma_inv = torch.argsort(sigma, dim=-1).to(device).type(default_tensor_type)
        y_inv = torch.argsort(y, dim=-1).to(device).type(default_tensor_type)
        sigma_pairs = (sigma_inv.unsqueeze(dim=-1) > sigma_inv.unsqueeze(dim=-2)).to(device).type(default_tensor_type)
        y_pairs = (y_inv.unsqueeze(dim=-1) > y_inv.unsqueeze(dim=-2)).to(device).type(default_tensor_type)
        res = torch.sum(torch.abs(sigma_pairs-y_pairs), dim=[-2,-1])
        if norm:
            return res/(n*(n-1))
        else:
            return res/2

In [ ]:
K = kendall_matrix(all_ranks)

In [ ]:
def get_all_buckets(all_ranks, n_items=4):
    list_bucket = list()

    for rank1 in all_ranks:
        for rank2 in all_ranks:
            if kendall_tau_numpy(rank1,rank2) == 1.0:
                list_bucket.append( [rank1,rank2] )
    
    for i in np.arange(4):
        temp_ranks = list()
        for rank in all_ranks:
            if rank[3] == i:
                temp_ranks.append(rank)
        list_bucket.append(temp_ranks)
        
    list_bucket.append([np.array([0,1,2,3]), np.array([0,1,3,2]),np.array([1,0,2,3]),np.array([1,0,3,2])])
    list_bucket.append([np.array([0,2,1,3]), np.array([0,2,3,1]),np.array([2,0,1,3]),np.array([2,0,3,1])])
    list_bucket.append([np.array([0,3,1,2]), np.array([0,3,2,1]),np.array([3,0,1,2]),np.array([3,0,2,1])])
    list_bucket.append([np.array([1,2,0,3]), np.array([1,2,3,0]),np.array([2,1,0,3]),np.array([2,1,3,0])]) 
    list_bucket.append([np.array([1,3,0,2]), np.array([1,3,2,0]),np.array([3,1,0,2]),np.array([3,1,2,0])])
    list_bucket.append([np.array([2,3,0,1]), np.array([2,3,1,0]),np.array([3,2,0,1]),np.array([3,2,1,0])])

    list_bucket.append(all_ranks)
    for rank in all_ranks:
        list_bucket.append(list([rank]))
    
    return list_bucket

list_bucket = get_all_buckets(all_ranks)

In [ ]:
def bucket_distrib(all_ranks, list_bucket):
    l_ = list()
    div = len(list_bucket)
    for a_ in all_ranks:
        count = 0
        for b_ in list_bucket:
            count += 1
            if (b_ == a_).all():
                l_.append(1.0/div)
                count += 1
                continue
        if count == div:
            l_.append(0)
    return np.matrix(l_)